Just generation with pretrained models

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os
import re
import sys
import json

Using TensorFlow backend.


In [2]:
def readfile(fname):
    with open(fname, 'r') as f:
        return f.read().lower()

def preprocess(poem):
    # left only words
    poem = re.sub('[^!а-яіїєА-ЯІЇЄ\s\,\.\-\—\:\n\!\(\)\?’]', ' ', poem)
    return poem.replace('\t', '\n')

def readjson(fname):
    with open(fname, 'r') as f:
        return json.load(f)

In [3]:
folder = './data/stus/'
file = 'Stus_Vasyl.Tom_3_1.Palimpsesty.1576.ua.txt'
raw_text = preprocess(readfile(folder + file))

char_to_int = readjson('./model/5/char_to_int.json')
int_to_char = readjson('./model/5/int_to_char.json')
int_to_char = {int(k):v for k,v in int_to_char.items()}

n_chars = len(raw_text)
n_vocab = len(char_to_int)

seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

In [5]:
!ls ./model/5

char_to_int.json  int_to_char.json  weights-improvement-79-1.8094.hdf5


In [8]:
model_file = 'model/5/weights-improvement-79-1.8094.hdf5'
model = Sequential()
model.add(LSTM(256, input_shape=(100, 1), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(44, activation='softmax'))
model.load_weights(model_file)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [9]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"{}\"".format(''.join([int_to_char[value] for value in pattern])))
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:]
    
print("\nDone.")

Seed:
"оцих запон,
в мені бо проросла зернина кожна
і кожна плідна, і туга й неложна
в собі зібгала крик ви"
соки сонце сон.
і скнце сонце повілий стожори,
на вітрі порога в світання
повідіні в світі — на вітрі почози
за себе відчути серце долоні,
до сонця — на вітрі почози — ти збагнеш
і від столо об собою на стіні,
до серце повго долоні — то від тиої
послухай на вітрі почози — ти стає,
та все тиття — на вітрі почожнечи,
на вітрі порога в світі за мертвий край.
на вітрі подобами своє сорики.
так порога — на вітрі почожнечі,
до серце повго долі — на вітрі,
на вітрі порога в світання
повідіні в світі — на вітрі почози
за себе відчути серце долони,
до сонця — на вітрі почози — ти збагнеш
і від столо об собою на стіні,
до серце повго долоні — то від тиої
послухай на вітрі почози — ти стає,
та все тиття — на вітрі почожнечи,
на вітрі порога в світі за мертвий край.
на вітрі подобами своє сорики.
так порога — на вітрі почожнечі,
до серце повго долі — на вітрі,
на вітрі порога в світання
пов

In [10]:
pattern_str = "гойдається вечора зламана віть"
pattern_str = " "*(100-len(pattern_str))+pattern_str
pattern = [char_to_int[value] for value in pattern_str]
print(pattern_str)
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:]
    
print("\nDone.")

                                                                      гойдається вечора зламана віть
.
на вітрі порога в світання —
за крайсвіту — на вітрі почожня
на вітрі за серце в світання
повідіні в світі — на вітрі почози
за себе відчути серце долоні,
до сонця — на вітрі почози — ти збагнеш
і від столо об собою на стіні,
до серце повго долоні — то від тиої
послухай на вітрі почози — ти стає,
та все тиття — на вітрі почожнечи,
на вітрі порога в світі за мертвий край.
на вітрі подобами своє сорики.
так порога — на вітрі почожнечі,
до серце повго долі — на вітрі,
на вітрі порога в світання
повідіні в світі — на вітрі почози
за себе відчути серце долони,
до сонця — на вітрі почози — ти збагнеш
і від столо об собою на стіні,
до серце повго долоні — то від тиої
послухай на вітрі почози — ти стає,
та все тиття — на вітрі почожнечи,
на вітрі порога в світі за мертвий край.
на вітрі подобами своє сорики.
так порога — на вітрі почожнечі,
до серце повго долі — на вітрі,
на вітрі порога в св